In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [16]:
outbound = pd.read_csv('../data/clean/LAX_Origin_Flight_Delays_2016.csv')
inbound = pd.read_csv('../data/clean/LAX_Dest_Flight_Delays_2016.csv')

#flight_list = [outbound_file, inbound_file]
#all_flights = pd.concat(flight_list)

In [17]:
total_outbound_flights = len(outbound['FlightNum'])
total_inbound_flights = len(inbound['FlightNum'])

In [29]:
outbound_delays = outbound.loc[outbound['DepDel15'] == 1]
total_delays = len(outbound_delays['FlightNum'])
print(f"Total outbound delays: {total_delays}")

monthly_outbound = outbound_delays.groupby(['Month']).sum()
print(monthly_outbound['DepDel15'].head(12))
print(monthly_outbound['DepDel15'].sum())

Total outbound delays: 48494
Month
1     3258.0
2     2401.0
3     3768.0
4     3053.0
5     3490.0
6     5513.0
7     5260.0
8     5227.0
9     3246.0
10    3968.0
11    3247.0
12    6063.0
Name: DepDel15, dtype: float64
48494.0
